In [44]:
# importing essential libraries 
import pandas as pd
import numpy as np 
import os 

In [45]:
os.getcwd()

'/Users/saaketvarma/Desktop/Rancho_Technical_Interview'

In [46]:
os.chdir('/Users/saaketvarma/Desktop/Rancho_Technical_Interview')

In [47]:
#open the excel files and the required sheets. 
xls = pd.ExcelFile('Technical Test - Data Wrangling.xlsx')
Patient_Clinical_Data = pd.read_excel(xls, 'Patient_clinical_data')
Tissue_Sample = pd.read_excel(xls, 'Tissue Sample Metadata')
Serum_Protein = pd.read_excel(xls, 'Serum Protein data')
RNA_seq = pd.read_excel(xls, 'RNA-seq (RPKM)')
Example_report = pd.read_excel(xls, 'Example report')

In [49]:
RNA_seq_t = RNA_seq.set_index("GeneID").T

In [50]:
#merge the RNA_seq data with Tissue sample data and include a column with Result units, along with reformatting the data
Merge_Tissue_Rna_seq = Tissue_Sample.merge(RNA_seq_t, right_index=True, left_on='Sample', 
                    how='left').drop(columns=['RIN', 
                    "Total Reads(millions)"]).melt(id_vars = ['Patient  Number', 'Sample', 'Sample type', 
                    'Material']).rename(columns={'variable':'Gene_Symbol', 'value':'Result', "Sample type":"Sample_General_Pathology", "Material":"Material_type"})
Merge_Tissue_Rna_seq["Result_Units"] = "RPKM"

In [51]:
# format the Serum protein data and include a column, "Material_type". 
Serum_Protein =Serum_Protein.rename(columns={"Patient": "Patient  Number"})
Serum_Protein["Material_type"] = "Serum"
Serum_Protein["Result_Units"] = "g/L"
Serum_Protein["Serum IL-6 Receptor (g/L)"] = Serum_Protein["Serum IL-6 Receptor (mg/L)"].apply(lambda x: pd.to_numeric(x, errors='coerce'))
Serum_Protein["Serum IL-6 Receptor (g/L)"] = Serum_Protein["Serum IL-6 Receptor (g/L)"]/1000
Serum_Protein["Serum IL-6 Receptor (g/L)"] =Serum_Protein["Serum IL-6 Receptor (g/L)"].fillna(Serum_Protein["Serum IL-6 Receptor (mg/L)"])
Serum_Protein = Serum_Protein[['Patient  Number', 'Sample', 'Serum IL-6 (g/L)', 'Serum IL-6 Receptor (g/L)','Material_type', 'Result_Units']].rename(columns={'Serum IL-6 (g/L)':'IL6', 'Serum IL-6 Receptor (g/L)':'IL6R'})
Serum_Protein= Serum_Protein.melt(id_vars = ['Patient  Number', 'Sample', 'Material_type', 'Result_Units']).rename(columns={'variable':'Gene_Symbol', 'value':'Result'})

In [52]:
# Merge between Patient_Clinical_Data and updated Merge_Tissue_Rna_seq data 
Merge1= Patient_Clinical_Data.merge(Merge_Tissue_Rna_seq, on='Patient  Number').rename(columns={'Patient  Number':'Patient_ID', 'Unique_Patient_ID':'Study_ID'})


In [53]:
Merge1.head()

,Study_ID,Patient_ID,Sex,Age,Sample,Sample_General_Pathology,Material_type,Gene_Symbol,Result,Result_Units
0,RO4532A,1001,M,41.67324,CRC1N,Normal,RNA,ICAM1,4.662774,RPKM
1,RO4532A,1001,M,41.67324,CRC1T,Liver Tumor,RNA,ICAM1,8.354152,RPKM
2,RO4532A,1001,M,41.67324,CRC1M,Metastic Lung,RNA,ICAM1,11.863796,RPKM
3,RO4532A,1001,M,41.67324,CRC1N,Normal,RNA,IL6,0.000000,RPKM
4,RO4532A,1001,M,41.67324,CRC1T,Liver Tumor,RNA,IL6,0.783566,RPKM


In [54]:
# Merge between Patient_Clinical_Data and and Serum_Protein data 
Merge2 = Patient_Clinical_Data.merge(Serum_Protein, on=['Patient  Number'], 
                                     how="inner").rename(columns={'Patient  Number':'Patient_ID',
                                                                  'Unique_Patient_ID':'Study_ID'})

In [55]:
# merge 1 + merge 2
Merge3 = pd.merge(Merge1, Merge2, on = ["Study_ID", "Patient_ID", "Sample", "Age", "Sex", "Material_type", "Gene_Symbol", "Result",'Result_Units'],
                 how= "outer")

In [65]:
Merge3.head(40)

,Study_ID,Patient_ID,Sex,Age,Sample,Sample_General_Pathology,Material_type,Gene_Symbol,Result,Result_Units
0,RO4532A,1001,M,41.673240,CRC1N,Normal,RNA,ICAM1,4.66277,RPKM
1,RO4532A,1001,M,41.673240,CRC1T,Liver Tumor,RNA,ICAM1,8.35415,RPKM
2,RO4532A,1001,M,41.673240,CRC1M,Metastic Lung,RNA,ICAM1,11.8638,RPKM
3,RO4532A,1001,M,41.673240,CRC1N,Normal,RNA,IL6,0,RPKM
4,RO4532A,1001,M,41.673240,CRC1T,Liver Tumor,RNA,IL6,0.783566,RPKM
5,RO4532A,1001,M,41.673240,CRC1M,Metastic Lung,RNA,IL6,0.874109,RPKM
6,RO4532A,1001,M,41.673240,CRC1N,Normal,RNA,IL6R,12.1537,RPKM
7,RO4532A,1001,M,41.673240,CRC1T,Liver Tumor,RNA,IL6R,3.11807,RPKM
8,RO4532A,1001,M,41.673240,CRC1M,Metastic Lung,RNA,IL6R,5.72837,RPKM
9,RO4532A,1001,M,41.673240,CRC1N,Normal,RNA,VCAM1,4.59635,RPKM


In [105]:
# formatting merge 3
Output = Merge3.sort_values(["Study_ID",'Patient_ID'],ascending = [False, True]).reset_index(drop=True)
Output["Unique_Patient_ID"] = Output.Study_ID + '_' + Output.Patient_ID.astype(str)
Output = Output[['Study_ID', 'Patient_ID', 'Unique_Patient_ID','Sex', 'Age', 'Sample','Sample_General_Pathology', 'Material_type', 'Gene_Symbol', 'Result', 'Result_Units']]
Output["Status"] = np.NAN
Output["Age"] = round(Output["Age"]).astype(int)
Output['Sex'].replace(['M', 'F'] ,['MALE', 'FEMALE'], inplace=True)
Output['Sample_General_Pathology'].replace(['Normal', 'Liver Tumor', 'Metastic Lung'] ,['NORMAL', 'PRIMARY', 'METASTIC'], inplace=True)
Output.Patient_ID = pd.to_numeric(Output.Patient_ID)
Output.Sample_General_Pathology = Output.Sample_General_Pathology.replace('', np.NAN)

In [106]:
Output.head()

,Study_ID,Patient_ID,Unique_Patient_ID,Sex,Age,Sample,Sample_General_Pathology,Material_type,Gene_Symbol,Result,Result_Units,Status
0,RO4532A,1001,RO4532A_1001,MALE,42,CRC1N,NORMAL,RNA,ICAM1,4.66277,RPKM,NaN
1,RO4532A,1001,RO4532A_1001,MALE,42,CRC1T,PRIMARY,RNA,ICAM1,8.35415,RPKM,NaN
2,RO4532A,1001,RO4532A_1001,MALE,42,CRC1M,METASTIC,RNA,ICAM1,11.8638,RPKM,NaN
3,RO4532A,1001,RO4532A_1001,MALE,42,CRC1N,NORMAL,RNA,IL6,0,RPKM,NaN
4,RO4532A,1001,RO4532A_1001,MALE,42,CRC1T,PRIMARY,RNA,IL6,0.783566,RPKM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,RO4532A,1105,RO4532A_1105,MALE,53,CRC5M,METASTIC,RNA,VCAM1,4.29993,RPKM,NaN
96,RO4532A,1105,RO4532A_1105,MALE,53,CRC5T,NORMAL,RNA,SELE,0.326909,RPKM,NaN
97,RO4532A,1105,RO4532A_1105,MALE,53,CRC5N,PRIMARY,RNA,SELE,0.749328,RPKM,NaN
98,RO4532A,1105,RO4532A_1105,MALE,53,CRC5M,METASTIC,RNA,SELE,0.211455,RPKM,NaN


In [57]:
#save as excel 
Output.to_excel("Data_Wrangle_output.xlsx")
Output.to_csv("Data_Wrangle_output.csv")